In [36]:
from scipy.linalg import fractional_matrix_power
def gcn(A):
    I = np.identity(A.shape[0]) #create Identity Matrix of A
    A_hat = A + I #add self-loop to A
    D = np.diag(np.sum(A_hat, axis=0)) #create Degree Matrix of A
    D_half_norm = fractional_matrix_power(D, -0.5) #calculate D to the power of -0.5
    eq = D_half_norm.dot(A_hat).dot(D_half_norm)
    return eq

In [47]:
import torch
import torch.nn as nn
class binaryClassification(nn.Module):
    def __init__(self, A):
        super(binaryClassification, self).__init__()
        # Number of input features is 12.
        g_out = 12
        self.A = gcn(A)
        self.double_a = gcn(np.matmul(self.A, self.A))
        self.A = torch.from_numpy(self.A).float()
        self.double_a = torch.from_numpy(self.double_a).float()
        self.layer_1 = nn.Linear(148, 64) 
        self.layer_2 = nn.Linear(64, 64)
        self.layer_out = nn.Linear(16 * 64, 1) 
        self.encode = nn.Sequential(
            nn.Linear(148, 64),
            nn.ReLU(),
            nn.BatchNorm1d(64),
        )
        self.lin = nn.Sequential(
            nn.Linear(3, 12),
            nn.ReLU(),
            nn.BatchNorm1d(64),
            nn.Linear(12, 12),
            nn.ReLU(),
            nn.BatchNorm1d(64),
            nn.Linear(12, 12),
            nn.ReLU(),
            nn.Linear(12, 16),
        )
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.1)
        self.batchnorm1 = nn.BatchNorm1d(64)
        self.batchnorm2 = nn.BatchNorm1d(64)
    def forward(self, inputs):
      #  x = self.encode(inputs)
        x = self.encode(inputs)
        two = torch.matmul(self.A, np.transpose(inputs))
        two = np.transpose(two)
        two = self.encode(two)
        three = torch.matmul(self.double_a, np.transpose(inputs))
        three = np.transpose(three)
        three = self.encode(three)
        x = torch.stack((x, two, three), dim = 2)
        x = self.lin(x).flatten(start_dim = 1)
        x = self.layer_out(x)
        return x

In [48]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter('runs/fashion_mnist_experiment_1')


In [49]:
# import torch
from torch.utils.data import Dataset, DataLoader
from torchsample.callbacks import EarlyStopping

class trainData(Dataset):
    
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data
        
    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]
        
    def __len__ (self):
        return len(self.X_data)
from input_data import load_data
import numpy as np
import sklearn
import sklearn.model_selection
import torch.optim as optim
A, f, labels, total = load_data()
A = A.toarray()
f = f.to_numpy()
f = np.transpose(f)
labels = np.array(labels)
labels = np.expand_dims(labels, axis=1)
tensor_input = torch.from_numpy(f).float()
tensor_label = torch.Tensor(labels).float()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")  




/Users/Mihir/Downloads/gae-master-2/gae/input_data.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  i[1]['label'] = l


In [50]:
def binary_acc(y_pred, y_test):
    y_pred_tag = torch.round(torch.sigmoid(y_pred))
    correct_results_sum = (y_pred_tag == y_test).sum().float()
    acc = correct_results_sum/y_test.shape[0]
    acc = torch.round(acc * 100)
    return acc

In [51]:
def split_data(index, k_fold, x, y):
    size = np.shape(x)[0]
    idx = range(size)
    r = range(0, int(size/k_fold))
    r = [i + index * int(size/k_fold) for i in r]
    idx = list(set(idx) - set(r))
    x_mb     = x[idx].astype(float)
    y_mb = y[idx].astype(float) 
    return x_mb, y_mb
def get_missing_data(index, k_fold, x, y):
    size = np.shape(x)[0]
    idx = range(size)
    r = range(0, int(size/k_fold))
    r = [i + index * int(size/k_fold) for i in r]
    idx = list(set(r))
    x_mb     = x[idx].astype(float)
    y_mb     = y[idx].astype(float)
    return x_mb, y_mb

In [52]:
x, y = split_data(0, 5, f, labels)
x_test, y_test = get_missing_data(0, 5, f, labels)

In [53]:
import pandas as pd
import sklearn.model_selection
import numpy as np
from input_data import load_data

In [54]:
import torch.utils.data


In [55]:
import math
k_folds = 5
total_acc = 0
for i in range(k_folds):
    criterion = nn.BCEWithLogitsLoss()
    model = binaryClassification(A)
    loss = criterion
    optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9)
    model.train()
    f_train, label_train = split_data(i, k_folds, f, labels)
    f_test, label_test = get_missing_data(i, k_folds, f, labels)
    train_data = trainData(torch.from_numpy(f_train).float(), torch.Tensor(label_train).float())
    train_loader = DataLoader(dataset=train_data, batch_size=300, shuffle=True)
    for e in range(1, 100):
        epoch_loss = 0
        epoch_acc = 0
        test_acc = 0
        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()

            y_pred = model(X_batch)

            loss = criterion(y_pred, y_batch)
            acc = binary_acc(y_pred, y_batch)

            test_acc += binary_acc(model(torch.Tensor(f_test).float()), torch.tensor(label_test).float())
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        if(epoch_loss/len(train_loader)<=0.5):
            break
        #print(f'Epoch {e+0:03}: | Loss: {epoch_loss/len(train_loader):.5f} | Acc: {epoch_acc/len(train_loader):.3f}|Test:{test_acc/len(train_loader):.5f}')
    print("Accuracy:" + str(roc_auc_score(label_test,torch.sigmoid(model(torch.Tensor(f_test).float())).detach().numpy())))
    total_acc += (roc_auc_score(label_test,torch.sigmoid(model(torch.Tensor(f_test).float())).detach().numpy()))


Accuracy:0.8830815298761399
Accuracy:0.8046070460704606
Accuracy:0.7848923785027752
Accuracy:0.8415411748745083
Accuracy:0.7893366228070176


In [56]:
print(total_acc/k_folds)

0.8206917504261803
